In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

cifar = keras.datasets.cifar10
(trainImages, trainLabels), (testImages, testLabels) = cifar.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2

#scaledTrainImages = []
#scaledTestImages = []

trainImages = (trainImages / 255.0) - .5
testImages = (testImages / 255.0) - .5

#for i in range(0, len(trainImages)):
#  scaledTrainImages.append(cv2.resize(trainImages[i], (64, 64)))

#for i in range(0, len(testImages)):
#  scaledTestImages.append(cv2.resize(testImages[i], (64, 64)))

#scaledTrainImages = np.array(scaledTrainImages)
#scaledTestImages = np.array(scaledTestImages)

trainLabels = keras.utils.to_categorical(trainLabels)
testLabels = keras.utils.to_categorical(testLabels)

In [0]:
savedModelDir = '/content/drive/My Drive/School Files/Senior/Fall/Machine Learning/finalProject/part2/models/'
def loadModel(path):
  return keras.models.load_model(path)

In [0]:
from random import randint
import matplotlib.pyplot as plt

labelMap = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

def classify(model):

  index = randint(0, len(testImages))

  image = testImages[index]
  label = testLabels[index]
  label = list(label)
  label = label.index(max(label))

  #print(image + .5)

  plt.figure(1)
  plt.xticks([])
  plt.yticks([])
  plt.title(labelMap[label])
  plt.imshow(image + .5)
  plt.show()

  #formattedImage = np.expand_dims(np.array(image), axis = 3)
  formattedImage = np.expand_dims(np.array(image), axis = 0)

  predictedLabel = list(model.predict(formattedImage))
  predictedLabel = list(predictedLabel[0])

  print(labelMap[predictedLabel.index(max(predictedLabel))])

In [0]:
def createModel():

  model = keras.models.Sequential()

  model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'elu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001), input_shape = (32, 32, 3)))
  model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.MaxPool2D())
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(256, (3, 3), padding = 'same', activation = 'elu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.Conv2D(256, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.MaxPool2D())
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'elu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'elu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.MaxPool2D())
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'elu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.Conv2D(512, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer = keras.regularizers.l2(.001)))
  model.add(keras.layers.MaxPool2D())
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.BatchNormalization())
  
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(128, activation = 'elu'))
  model.add(keras.layers.Dense(64, activation = 'elu'))
  model.add(keras.layers.Dense(10, activation = 'softmax'))

  return model


In [0]:
def lrSchedule(epochs):
  if(epochs < 15):
    return .0001
  if(epochs >= 15 or epochs < 35):
    return .00001
  if(epochs >= 35 or epochs < 65):
    return .000001
  else:
    return .0000001

In [0]:
saveDir = '/content/models'
modelDir = saveDir + '/model.{epoch:02d}-{val_acc:.2f}.hdf5'

if os.path.exists(saveDir) == False:
  os.mkdir(saveDir)

#for f in os.listdir(saveDir):
  #os.remove(saveDir + '/' + f)

saveCallback = keras.callbacks.ModelCheckpoint(filepath = modelDir, monitor = 'val_acc', save_best_only = True)
lrSchedulerCallback = keras.callbacks.LearningRateScheduler(lrSchedule)
adam = keras.optimizers.Adam(learning_rate = 0.0001)
sgd = keras.optimizers.SGD(learning_rate = .1)

cnn = createModel()
cnn.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['acc'])

In [0]:
cnn.fit(trainImages, trainLabels, batch_size = 128, epochs = 500, callbacks = [saveCallback], validation_data = (testImages, testLabels))

In [26]:
cnn.evaluate(testImages, testLabels)

10000/10000 [==============================] - 6s 643us/sample - loss: 1.0083 - acc: 0.8525


[1.0082694493293762, 0.8525]

In [31]:
cnn = loadModel(savedModelDir + 'model.77-0.86.hdf5')
cnn.summary()
#keras.utils.plot_model(cnn, saveDir + '/modelDiagram.png', show_shapes = True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 32, 128)       3584      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 256)      